# 0. Data Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Dacon_RF_parameter_tuning

/content/drive/MyDrive/Dacon_RF_parameter_tuning


In [ ]:
ls

 0325_EDA_Gridsearch.ipynb   EDA.ipynb               submit/          'test_grid.ipynb의 사본'
 0326_Generalization.ipynb   grid_test2.ipynb        submit.ipynb      train.csv
 0327_RandomSearch.ipynb     sample_submission.csv   test_grid.ipynb


In [ ]:
# 대회 제공 시드로 고정
import random
random.seed(42)

In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame

In [ ]:
train = pd.read_csv('train.csv')
train.head()

,person_id,Sex,past_login_total,past_1_month_login,past_1_week_login,sub_size,email_type,phone_rat,apple_rat,login
0,0,0,3.0,1.0,1.0,0.0,0,0.000000,0.000000,1
1,1,1,111.0,26.0,7.0,2.0,0,0.072072,0.000000,1
2,3,1,13.0,13.0,11.0,7.0,0,0.076923,1.000000,1
3,4,1,28.0,12.0,5.0,0.0,0,0.071429,0.071429,1
4,5,1,4.0,4.0,4.0,0.0,2,0.000000,0.000000,1


# 1. Random_Search
- 편향된 데이터에서 준수한 성능을 보이는 Random_Search 시행
- Random_Search란, 주어진 구간 안에서 랜덤으로 숫자를 뽑아 하이퍼파라미터로 실험하는 것.
- Scikit-learn의 RandomSearchCV()사용

In [ ]:
#제출1
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# 학습 데이터 불러오기 1
train_data = pd.read_csv('train.csv')

# RandomForestClassifier 모델 정의
rf_model = RandomForestClassifier()

# 탐색할 하이퍼파라미터 범위 지정
random_search  = {
    'n_estimators': [100, 200, 300, 400, 500, 600, 700,800], # 10 ~ 1000 사이의 양의 정수
    'criterion': ['gini','entropy'], # 옵션: 'gini', 'entropy'
    'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90,100, 110], # None 또는 양의 정수
    'min_samples_split': [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16, 17,18,19,20], # 2 이상의 정수 또는 0과 1 사이의 실수
    'min_samples_leaf': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16, 17,18,19,20], # 1 이상의 정수 또는 0과 0.5 사이의 실수 (비율을 나타냄, (0, 0.5] )
     'min_weight_fraction_leaf':[0.0, 0.1, 0.2, 0.3, 0.4,0.5], # 0.0에서 0.5 사이의 실수
    'max_features': ['auto'], # 옵션: 'auto', 'sqrt', 'log2', None 또는 양의 정수/실수
     'max_leaf_nodes' : [None, 0, 1, 2, 3] , # None 또는 양의 정수 # 0.0 이상의 실수
     'min_impurity_decrease' : [0.01, 0.005],
    'bootstrap': [True, False] # 옵션: True, False.
}

clf = RandomForestClassifier()

# GridSearchCV 실행
model = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 240,
                               cv = 4, verbose= 1, random_state= 42, n_jobs = -1)
model.fit(train_data.drop('login', axis=1), train_data['login'])


#rand_search = RandomSearchCV(rf_model, n_iter=50, param_distribution=random_search , random_state=42))
#rand_search.fit(train_data.drop('login', axis=1), train_data['login']) # X, y
best_params = model.best_params_
best_score = model.best_score_

print(best_params)
print(best_score)

Fitting 4 folds for each of 240 candidates, totalling 960 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
388 fits failed out of a total of 960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
196 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ut

{'n_estimators': 300, 'min_weight_fraction_leaf': 0.3, 'min_samples_split': 15, 'min_samples_leaf': 3, 'min_impurity_decrease': 0.005, 'max_leaf_nodes': 3, 'max_features': 'auto', 'max_depth': 110, 'criterion': 'gini', 'bootstrap': False}
0.8884654471544715


In [ ]:
#제출2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# 학습 데이터 불러오기 1
train_data = pd.read_csv('train.csv')

# RandomForestClassifier 모델 정의
rf_model = RandomForestClassifier()

# 탐색할 하이퍼파라미터 범위 지정
random_search  = {
    'n_estimators': [100, 200, 300, 400, 500, 600, 700,800], # 10 ~ 1000 사이의 양의 정수
    'criterion': ['gini','entropy'], # 옵션: 'gini', 'entropy'
    'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90,100, 110], # None 또는 양의 정수
    'min_samples_split': [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16, 17,18,19,20], # 2 이상의 정수 또는 0과 1 사이의 실수
    'min_samples_leaf': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16, 17,18,19,20], # 1 이상의 정수 또는 0과 0.5 사이의 실수 (비율을 나타냄, (0, 0.5] )
     'min_weight_fraction_leaf':[0.0, 0.1, 0.2, 0.3, 0.4,0.5], # 0.0에서 0.5 사이의 실수
    'max_features': ['auto','sqrt', 'log2', None], # 옵션: 'auto', 'sqrt', 'log2', None 또는 양의 정수/실수
     'max_leaf_nodes' : [None, 0, 1, 2, 3] , # None 또는 양의 정수 # 0.0 이상의 실수
     'min_impurity_decrease' : [0.0, 0.01, 0.005],
    'bootstrap': [True, False] # 옵션: True, False.
}

clf = RandomForestClassifier()

# GridSearchCV 실행
model = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 120,
                               cv = 4, verbose= 1, random_state= 42, n_jobs = -1)
model.fit(train_data.drop('login', axis=1), train_data['login'])


#rand_search = RandomSearchCV(rf_model, n_iter=50, param_distribution=random_search , random_state=42))
#rand_search.fit(train_data.drop('login', axis=1), train_data['login']) # X, y
best_params = model.best_params_
best_score = model.best_score_

print(best_params)
print(best_score)

Fitting 4 folds for each of 120 candidates, totalling 480 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
148 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
84 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/uti

{'n_estimators': 500, 'min_weight_fraction_leaf': 0.5, 'min_samples_split': 20, 'min_samples_leaf': 16, 'min_impurity_decrease': 0.0, 'max_leaf_nodes': None, 'max_features': 'sqrt', 'max_depth': 70, 'criterion': 'entropy', 'bootstrap': True}
0.8884654471544715


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# 학습 데이터 불러오기 1
train_data = pd.read_csv('train.csv')

# RandomForestClassifier 모델 정의
rf_model = RandomForestClassifier()

# 탐색할 하이퍼파라미터 범위 지정
random_search  = {
    'n_estimators': [100, 200, 300, 400, 500, 600, 700,800], # 10 ~ 1000 사이의 양의 정수
    'criterion': ['gini','entropy'], # 옵션: 'gini', 'entropy'
    'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90,100, 110], # None 또는 양의 정수
    'min_samples_split': [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16, 17,18,19,20], # 2 이상의 정수 또는 0과 1 사이의 실수
    'min_samples_leaf': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16, 17,18,19,20], # 1 이상의 정수 또는 0과 0.5 사이의 실수 (비율을 나타냄, (0, 0.5] )
     'min_weight_fraction_leaf':[0.0, 0.1, 0.2, 0.3, 0.4,0.5], # 0.0에서 0.5 사이의 실수
    'max_features': ['auto','sqrt', 'log2', None], # 옵션: 'auto', 'sqrt', 'log2', None 또는 양의 정수/실수
     'max_leaf_nodes' : [None, 0, 1, 2, 3] , # None 또는 양의 정수 # 0.0 이상의 실수
     'min_impurity_decrease' : [0.0, 0.01, 0.005],
    'bootstrap': [True, False] # 옵션: True, False.
}

clf = RandomForestClassifier()

# GridSearchCV 실행
model = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 360,
                               cv = 4, verbose= 1, random_state= 42, n_jobs = -1)
model.fit(train_data.drop('login', axis=1), train_data['login'])


#rand_search = RandomSearchCV(rf_model, n_iter=50, param_distribution=random_search , random_state=42))
#rand_search.fit(train_data.drop('login', axis=1), train_data['login']) # X, y
best_params = model.best_params_
best_score = model.best_score_

print(best_params)
print(best_score)

Fitting 4 folds for each of 360 candidates, totalling 1440 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
544 fits failed out of a total of 1440.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
276 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/u

{'n_estimators': 500, 'min_weight_fraction_leaf': 0.5, 'min_samples_split': 20, 'min_samples_leaf': 16, 'min_impurity_decrease': 0.0, 'max_leaf_nodes': None, 'max_features': 'sqrt', 'max_depth': 70, 'criterion': 'entropy', 'bootstrap': True}
0.8884654471544715


#2. 제출

In [ ]:
# 제출1
df = {'n_estimators':300,
      'criterion' : 'gini',
      'max_depth' : 110,
      'min_samples_split' : 15,
      'min_samples_leaf' : 3,
      'min_weight_fraction_leaf' : 0.3,
      'max_features' : 'auto',
      'max_leaf_nodes' : 3,
      'min_impurity_decrease' : 0.005,
      'bootstrap': False
      }
df = DataFrame(df, index=['a'])
df
df.to_csv("./submit0327_1.csv", index=False)

In [ ]:
# 찐 제출2
df = {'n_estimators':500,
      'criterion' : 'gini',
      'max_depth' : 2,
      'min_samples_split' : 40,
      'min_samples_leaf' : 30,
      'min_weight_fraction_leaf' : 0,
      'max_features' : 3,
      'max_leaf_nodes' : 2,
      'min_impurity_decrease' : 0.0,
      'bootstrap': False
      }
df = DataFrame(df, index=['a'])
df
df.to_csv("./submit0327_2.csv", index=False)

In [ ]:
# 찐 제출3 - 최고점수
df = {'n_estimators':500,
      'criterion' : 'gini',
      'max_depth' : 5,
      'min_samples_split' : 50,
      'min_samples_leaf' : 50,
      'min_weight_fraction_leaf' : 0,
      'max_features' : 3,
      'max_leaf_nodes' : None,
      'min_impurity_decrease' : 0.0,
      'bootstrap': False
      }
df = DataFrame(df, index=['a'])
df
df.to_csv("./submit0327_3.csv", index=False)

In [ ]:
# 제출4
df = {'n_estimators':500,
      'criterion' : 'gini',
      'max_depth' : 5,
      'min_samples_split' : 45,
      'min_samples_leaf' : 45,
      'min_weight_fraction_leaf' : 0,
      'max_features' : 3,
      'max_leaf_nodes' : None,
      'min_impurity_decrease' : 0.0,
      'bootstrap': False
      }
df = DataFrame(df, index=['a'])
df
df.to_csv("./submit0327_4.csv", index=False)

In [ ]:
df = {'n_estimators':500,
      'criterion' : 'gini',
      'max_depth' : 5,
      'min_samples_split' : 50,
      'min_samples_leaf' : 50,
      'min_weight_fraction_leaf' : 0,
      'max_features' : 3,
      'max_leaf_nodes' : 40,
      'min_impurity_decrease' : 0.0,
      'bootstrap': False
      }
df = DataFrame(df, index=['a'])
df
df.to_csv("./submit0327_5.csv", index=False)

# 3. 제출 결과
- 제출1, 2
 - random search 결과로 선택된 파라미터로 제출
 - 제출 1 score : 0.5732199367 -> 매우 낮음
 - 제출 2 score : 0.7261075949
 - random search한 결과가 이전 score보다 낮은 score 기록
- 제출3
 - 가장 높은 순위였던 submit0326_4에서, 직접 숫자를 조금씩 수정하며  성능 평가
 - score : 0.7935917722
 - 최고점수 기록

# 그 외 test


In [ ]:
# test 중 점수 파라미터 조합
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 종속변수(target)의 컬럼을 target으로의 선언이 필요합니다.
target = train['login']
X_train, X_test, y_train, y_test = train_test_split(train, target,
                                                      test_size=0.2,
                                                      shuffle=True,
                                                      stratify=target)

#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
rf_clf1 = RandomForestClassifier(n_estimators = 500, # 크면 성능 좋고 오래걸림?
                                 criterion = 'gini', # 보통은 둘 다 비슷, 특정 데이터에서는 차이나기도 함
                                max_depth = 5, # 작으면 과적합 방지
                                min_samples_split = 30, # 작을수록 과적합 가능성 높음
                                 min_samples_leaf = 30, # 데이터 양이 불균형하면 작게 설정 -> 크면 과적합 방지
                                 min_weight_fraction_leaf = 0, # 작아야됨
                                 max_features = 3,  # 편향된 데이터에서는 작아야됨
                                max_leaf_nodes = None, # 작아야됨
                                 min_impurity_decrease = 0, # 작아야됨
                                bootstrap = True) # 편향된 데이터에서는 False


rf_clf1.fit(X_train, y_train)
test_pred = rf_clf1.predict(X_test)
train_pred = rf_clf1.predict(X_train)
print('train 예측 정확도: {:.4f}'.format(accuracy_score(y_train,train_pred)))
print('test 예측 정확도: {:.4f}'.format(accuracy_score(y_test,test_pred)))

train 예측 정확도: 1.0000
test 예측 정확도: 1.0000


In [ ]:
# test 중 점수 파라미터 조합
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 종속변수(target)의 컬럼을 target으로의 선언이 필요합니다.
target = train['login']
X_train, X_test, y_train, y_test = train_test_split(train, target,
                                                      test_size=0.2,
                                                      shuffle=True,
                                                      stratify=target)

#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
rf_clf1 = RandomForestClassifier(n_estimators = 500, # 크면 성능 좋고 오래걸림?
                                 criterion = 'gini', # 보통은 둘 다 비슷, 특정 데이터에서는 차이나기도 함
                                max_depth = 5, # 작으면 과적합 방지
                                min_samples_split = 30, # 작을수록 과적합 가능성 높음
                                 min_samples_leaf = 30, # 데이터 양이 불균형하면 작게 설정 -> 크면 과적합 방지
                                 min_weight_fraction_leaf = 0, # 작아야됨
                                 max_features = 2,  # 편향된 데이터에서는 작아야됨
                                max_leaf_nodes = 2, # 작아야됨
                                 min_impurity_decrease = 0, # 작아야됨
                                bootstrap = True) # 편향된 데이터에서는 False


rf_clf1.fit(X_train, y_train)
test_pred = rf_clf1.predict(X_test)
train_pred = rf_clf1.predict(X_train)
print('train 예측 정확도: {:.4f}'.format(accuracy_score(y_train,train_pred)))
print('test 예측 정확도: {:.4f}'.format(accuracy_score(y_test,test_pred)))

train 예측 정확도: 0.9169
test 예측 정확도: 0.9160


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 종속변수(target)의 컬럼을 target으로의 선언이 필요합니다.
target = train['login']
X_train, X_test, y_train, y_test = train_test_split(train, target,
                                                      test_size=0.2,
                                                      shuffle=True,
                                                      stratify=target)

#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
rf_clf1 = RandomForestClassifier(n_estimators = 500, # 크면 성능 좋고 오래걸림?
                                 criterion = 'gini',
                                max_depth = 5, # 작으면 과적합 방지
                                min_samples_split = 50, # 작을수록 과적합 가능성 높음
                                 min_samples_leaf = 50, # 데이터 양이 불균형하면 작게 설정 -> 크면 과적합 방지
                                 min_weight_fraction_leaf = 0,
                                 max_features = 3, # 깊이 깊어지면 과적합 가능성 있음
                                max_leaf_nodes = None,
                                 min_impurity_decrease = 0,
                                bootstrap = True)


rf_clf1.fit(X_train, y_train)
test_pred = rf_clf1.predict(X_test)
train_pred = rf_clf1.predict(X_train)
print('train 예측 정확도: {:.4f}'.format(accuracy_score(y_train,train_pred)))
print('test 예측 정확도: {:.4f}'.format(accuracy_score(y_test,test_pred)))

train 예측 정확도: 0.9494
test 예측 정확도: 0.9389


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 종속변수(target)의 컬럼을 target으로의 선언이 필요합니다.
target = train['login']
X_train, X_test, y_train, y_test = train_test_split(train, target,
                                                      test_size=0.2,
                                                      shuffle=True,
                                                      stratify=target)

#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
rf_clf1 = RandomForestClassifier(n_estimators = 500, # 크면 성능 좋고 오래걸림?
                                 criterion = 'gini',
                                max_depth = 5, # 작으면 과적합 방지
                                min_samples_split = 50, # 작을수록 과적합 가능성 높음
                                 min_samples_leaf = 50, # 데이터 양이 불균형하면 작게 설정 -> 크면 과적합 방지
                                 min_weight_fraction_leaf = 0,
                                 max_features = 3, # 깊이 깊어지면 과적합 가능성 있음
                                max_leaf_nodes = 60,
                                 min_impurity_decrease = 0,
                                bootstrap = True)


rf_clf1.fit(X_train, y_train)
test_pred = rf_clf1.predict(X_test)
train_pred = rf_clf1.predict(X_train)
print('train 예측 정확도: {:.4f}'.format(accuracy_score(y_train,train_pred)))
print('test 예측 정확도: {:.4f}'.format(accuracy_score(y_test,test_pred)))

train 예측 정확도: 0.9589
test 예측 정확도: 0.9504


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 종속변수(target)의 컬럼을 target으로의 선언이 필요합니다.
target = train['login']
X_train, X_test, y_train, y_test = train_test_split(train, target,
                                                      test_size=0.2,
                                                      shuffle=True,
                                                      stratify=target)

#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
rf_clf1 = RandomForestClassifier(n_estimators = 500, # 크면 성능 좋고 오래걸림?
                                 criterion = 'gini',
                                max_depth = 5, # 작으면 과적합 방지
                                min_samples_split = 50, # 작을수록 과적합 가능성 높음
                                 min_samples_leaf = 50, # 데이터 양이 불균형하면 작게 설정 -> 크면 과적합 방지
                                 min_weight_fraction_leaf = 0,
                                 max_features = 3, # 깊이 깊어지면 과적합 가능성 있음
                                max_leaf_nodes = 40,
                                 min_impurity_decrease = 0,
                                bootstrap = False)


rf_clf1.fit(X_train, y_train)
test_pred = rf_clf1.predict(X_test)
train_pred = rf_clf1.predict(X_train)
print('train 예측 정확도: {:.4f}'.format(accuracy_score(y_train,train_pred)))
print('test 예측 정확도: {:.4f}'.format(accuracy_score(y_test,test_pred)))

train 예측 정확도: 1.0000
test 예측 정확도: 0.9962


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 종속변수(target)의 컬럼을 target으로의 선언이 필요합니다.
target = train['login']
X_train, X_test, y_train, y_test = train_test_split(train, target,
                                                      test_size=0.2,
                                                      shuffle=True,
                                                      stratify=target)

#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
rf_clf1 = RandomForestClassifier(n_estimators = 500, # 크면 성능 좋고 오래걸림?
                                 criterion = 'gini',
                                max_depth = 5, # 작으면 과적합 방지
                                min_samples_split = 45, # 작을수록 과적합 가능성 높음
                                 min_samples_leaf = 45, # 데이터 양이 불균형하면 작게 설정 -> 크면 과적합 방지
                                 min_weight_fraction_leaf = 0,
                                 max_features = 3, # 깊이 깊어지면 과적합 가능성 있음
                                max_leaf_nodes = None,
                                 min_impurity_decrease = 0,
                                bootstrap = True)


rf_clf1.fit(X_train, y_train)
test_pred = rf_clf1.predict(X_test)
train_pred = rf_clf1.predict(X_train)
print('train 예측 정확도: {:.4f}'.format(accuracy_score(y_train,train_pred)))
print('test 예측 정확도: {:.4f}'.format(accuracy_score(y_test,test_pred)))

train 예측 정확도: 0.9580
test 예측 정확도: 0.9580


In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 종속변수(target)의 컬럼을 target으로의 선언이 필요합니다.
target = train['login']
X_train, X_test, y_train, y_test = train_test_split(train, target,
                                                      test_size=0.2,
                                                      shuffle=True,
                                                      stratify=target)

#위의 결과로 나온 최적 하이퍼 파라미터로 다시 모델을 학습하여 테스트 세트 데이터에서 예측 성능을 측정
rf_clf1 = RandomForestClassifier(n_estimators = 500, # 크면 성능 좋고 오래걸림?
                                 criterion = 'gini',
                                max_depth = 5, # 작으면 과적합 방지
                                min_samples_split = 45, # 작을수록 과적합 가능성 높음
                                 min_samples_leaf = 45, # 데이터 양이 불균형하면 작게 설정 -> 크면 과적합 방지
                                 min_weight_fraction_leaf = 0,
                                 max_features = 3, # 깊이 깊어지면 과적합 가능성 있음
                                max_leaf_nodes = 20,
                                 min_impurity_decrease = 0,
                                bootstrap = True)


rf_clf1.fit(X_train, y_train)
test_pred = rf_clf1.predict(X_test)
train_pred = rf_clf1.predict(X_train)
print('train 예측 정확도: {:.4f}'.format(accuracy_score(y_train,train_pred)))
print('test 예측 정확도: {:.4f}'.format(accuracy_score(y_test,test_pred)))

train 예측 정확도: 0.9599
test 예측 정확도: 0.9504
